# StorageClasses

A **StorageClass** defines *how* dynamic storage is provisioned for PVCs (what backend, what performance, what reclaim behavior).

If you don’t define this explicitly and you rely on the **cluster default**, migrations between clusters/clouds often break (PVCs stay Pending or get the wrong storage type).


## Why it is used
- Make storage provisioning **portable and explicit**.
- Offer multiple storage tiers (fast SSD vs cheap HDD).
- Control policies like volume expansion and reclaim behavior.
- Avoid surprises when moving clusters (defaults differ).


## How it relates to PVCs
```text
PVC (request) -> StorageClass (how) -> PV (actual volume)
```

- A PVC can set `storageClassName: standard`.
- If `storageClassName` is omitted, Kubernetes may use the cluster’s **default StorageClass**.
- If there is no default (or the provisioner isn’t installed), the PVC can remain **Pending**.


## YAML template (generic)
StorageClasses are tied to a CSI driver/provisioner. The exact `provisioner` and `parameters` depend on your environment.

```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: standard
provisioner: example.csi.driver
reclaimPolicy: Delete
volumeBindingMode: WaitForFirstConsumer
allowVolumeExpansion: true
parameters:
  type: fast
```


## PVC that references a StorageClass explicitly
```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: app-data
spec:
  storageClassName: standard
  accessModes: ["ReadWriteOnce"]
  resources:
    requests:
      storage: 10Gi
```


## Pitfalls
- Relying on a **default StorageClass** is a portability trap.
- `volumeBindingMode` impacts scheduling: `WaitForFirstConsumer` is often safer to avoid cross-zone issues.
- Not all backends support `allowVolumeExpansion`.

## References
- StorageClasses: https://kubernetes.io/docs/concepts/storage/storage-classes/
- CSI drivers: https://kubernetes-csi.github.io/docs/
